In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests


lda = pd.read_csv('/content/speeches_spicy_with_lda_score.csv')
gmi = pd.read_csv('/content/Экономика_КНДР - Лист1-3.csv')


lda['Year'] = pd.to_datetime(lda['Date'], dayfirst=True).dt.year
lda_yearly = lda.groupby('Year')['lda_militarization_score'].mean().reset_index()


score_2006 = lda_yearly.loc[lda_yearly['Year'] == 2006, 'lda_militarization_score'].values[0]
score_2008 = lda_yearly.loc[lda_yearly['Year'] == 2008, 'lda_militarization_score'].values[0]
score_2007 = (score_2006 + score_2008) / 2

lda_yearly = pd.concat([
    lda_yearly,
    pd.DataFrame({'Year': [2007], 'lda_militarization_score': [score_2007]})
]).sort_values('Year').reset_index(drop=True)


gmi['Year'] = gmi['Date'].astype(str).str.extract(r'(\d{4})').astype(int)
gmi_clean = gmi[['Year', 'Реконструированный GMI']].copy()
gmi_clean = gmi_clean.dropna()
gmi_clean['Реконструированный GMI'] = gmi_clean['Реконструированный GMI'].astype(str).str.replace(',', '.').astype(float)


df = pd.merge(lda_yearly, gmi_clean, on='Year').sort_values('Year').reset_index(drop=True)


df['lda_smoothed'] = df['lda_militarization_score'].rolling(window=3, center=True).mean()
df['GMI_smoothed'] = df['Реконструированный GMI'].rolling(window=3, center=True).mean()


df_smooth = df.dropna(subset=['lda_smoothed', 'GMI_smoothed']).reset_index(drop=True)

df_smooth_diff = df_smooth.set_index('Year')[['GMI_smoothed', 'lda_smoothed']].diff().dropna()

def test_granger(data, max_lag=3):
    for lag in range(1, max_lag+1):
        print(f"\n Лаг = {lag}\n")
        print("Тест: влияет ли милитаризация на GMI?\n")
        grangercausalitytests(data[['GMI_smoothed', 'lda_smoothed']], maxlag=lag, verbose=True)
        print("\nТест: влияет ли GMI на милитаризацию?\n")
        grangercausalitytests(data[['lda_smoothed', 'GMI_smoothed']], maxlag=lag, verbose=True)

print(" Годы для анализа (после сглаживания и дифференцирования):", df_smooth_diff.index.tolist())
print(f" Используемое количество наблюдений: {df_smooth_diff.shape[0]}")


test_granger(df_smooth_diff, max_lag=3)

 Годы для анализа (после сглаживания и дифференцирования): [1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
 Используемое количество наблюдений: 25

 Лаг = 1

Тест: влияет ли милитаризация на GMI?


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0236  , p=0.8794  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=0.0270  , p=0.8696  , df=1
likelihood ratio test: chi2=0.0269  , p=0.8696  , df=1
parameter F test:         F=0.0236  , p=0.8794  , df_denom=21, df_num=1

Тест: влияет ли GMI на милитаризацию?


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0429  , p=0.8379  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=0.0491  , p=0.8247  , df=1
likelihood ratio test: chi2=0.0490  , p=0.8248  , df=1
parameter F test:         F=0.0429  , p=0.8379  , df_denom=21, df_num=1

 Лаг = 2

Тест: влияет ли милитаризация на GMI?


Granger C

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests


tfidf = pd.read_csv('/content/speeches_with_militarization-3.csv')

tfidf['Year'] = pd.to_datetime(tfidf['Date'], dayfirst=True).dt.year
tfidf_yearly = tfidf.groupby('Year')['militarization_score'].mean().reset_index()


if 2007 not in tfidf_yearly['Year'].values:
    score_2006 = tfidf_yearly.loc[tfidf_yearly['Year'] == 2006, 'militarization_score'].values[0]
    score_2008 = tfidf_yearly.loc[tfidf_yearly['Year'] == 2008, 'militarization_score'].values[0]
    score_2007 = (score_2006 + score_2008) / 2
    tfidf_yearly = pd.concat([
        tfidf_yearly,
        pd.DataFrame({'Year': [2007], 'militarization_score': [score_2007]})
    ]).sort_values('Year').reset_index(drop=True)

gmi = pd.read_csv('/content/Экономика_КНДР - Лист1-3.csv')


gmi['Year'] = gmi['Date'].astype(str).str.extract(r'(\d{4})').astype(int)


gmi_clean = gmi[['Year', 'Реконструированный GMI']].copy()
gmi_clean = gmi_clean.dropna()
gmi_clean['Реконструированный GMI'] = gmi_clean['Реконструированный GMI'].astype(str).str.replace(',', '.').astype(float)


df = pd.merge(tfidf_yearly, gmi_clean, on='Year').sort_values('Year').reset_index(drop=True)


df['tfidf_smoothed'] = df['militarization_score'].rolling(window=3, center=True).mean()
df['GMI_smoothed'] = df['Реконструированный GMI'].rolling(window=3, center=True).mean()


df_smooth = df.dropna(subset=['tfidf_smoothed', 'GMI_smoothed']).reset_index(drop=True)


df_smooth_diff = df_smooth.set_index('Year')[['GMI_smoothed', 'tfidf_smoothed']].diff().dropna()

print("Годы для анализа (после сглаживания и дифференцирования):", df_smooth_diff.index.tolist())
print(f"Используемое количество наблюдений: {df_smooth_diff.shape[0]}")


def test_granger(data, max_lag=3):
    for lag in range(1, max_lag + 1):
        print(f"\n Лаг = {lag}\n")
        print("Тест: влияет ли милитаризация (TF-IDF) на GMI?\n")
        grangercausalitytests(data[['GMI_smoothed', 'tfidf_smoothed']], maxlag=lag, verbose=True)
        print("\nТест: влияет ли GMI на милитаризацию (TF-IDF)?\n")
        grangercausalitytests(data[['tfidf_smoothed', 'GMI_smoothed']], maxlag=lag, verbose=True)


test_granger(df_smooth_diff, max_lag=3)

Годы для анализа (после сглаживания и дифференцирования): [1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
Используемое количество наблюдений: 25

 Лаг = 1

Тест: влияет ли милитаризация (TF-IDF) на GMI?


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.9815  , p=0.3331  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=1.1217  , p=0.2896  , df=1
likelihood ratio test: chi2=1.0963  , p=0.2951  , df=1
parameter F test:         F=0.9815  , p=0.3331  , df_denom=21, df_num=1

Тест: влияет ли GMI на милитаризацию (TF-IDF)?


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.9964  , p=0.1723  , df_denom=21, df_num=1
ssr based chi2 test:   chi2=2.2816  , p=0.1309  , df=1
likelihood ratio test: chi2=2.1795  , p=0.1399  , df=1
parameter F test:         F=1.9964  , p=0.1723  , df_denom=21, df_num=1

 Лаг = 2

Тест: влияет ли милитаризация (TF

/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print resul